In [ ]:
from qiskit_aer import AerSimulator

from qlbm.components import (
    CQLBM,
    CollisionlessInitialConditions,
    EmptyPrimitive,
    GridMeasurement,
)
from qlbm.infra import QiskitRunner, SimulationConfig
from qlbm.lattice import CollisionlessLattice
from qlbm.tools.utils import create_directory_and_parents

In [ ]:
# Load example with mixed boundary conditions and create output directory
lattice = CollisionlessLattice(
    {
        "lattice": {"dim": {"x": 16, "y": 16}, "velocities": {"x": 4, "y": 4}},
        "geometry": [
            {"shape": "cuboid", "x": [9, 12], "y": [3, 6], "boundary": "specular"},
            {"shape": "cuboid", "x": [9, 12], "y": [9, 12], "boundary": "bounceback"},
        ],
    }
)


output_dir = "qlbm-output/collisionless-2d-16x16-2-obstacle-mixed-qiskit"
create_directory_and_parents(output_dir)

In [ ]:
# Number of shots to simulate for each timestep when running the circuit
NUM_SHOTS = 2**12
# Number of timesteps to simulate
NUM_STEPS = 20

In [ ]:
# In the simulation configuration function the user can determine the specifications of the run
cfg = SimulationConfig(
    initial_conditions=CollisionlessInitialConditions(lattice),
    algorithm=CQLBM(lattice),
    postprocessing=EmptyPrimitive(lattice),
    measurement=GridMeasurement(lattice),
    target_platform="QISKIT",
    compiler_platform="QISKIT",
    optimization_level=0,
    statevector_sampling=True,
    execution_backend=AerSimulator(method="statevector"),
    sampling_backend=AerSimulator(method="statevector"),
)

In [ ]:
cfg.prepare_for_simulation()

In [ ]:
# Create a runner object to simulate the circuit
runner = QiskitRunner(
    cfg,
    lattice,
)

# Simulate the circuits using both snapshots
runner.run(
    NUM_STEPS,  # Number of time steps
    NUM_SHOTS,  # Number of shots per time step
    output_dir,
    statevector_snapshots=True,
)